In [1]:
import pandas as pd
import numpy as np
import os
from cassandra.cluster import Cluster
from datetime import datetime
import plotly.graph_objects as go
import settrade_v2
from settrade_v2 import Investor
import time

In [2]:
cluster = Cluster(['127.0.0.1']) 
session = cluster.connect()

In [3]:
import settrade_v2
from settrade_v2 import Investor

investor = Investor(
    app_id="atBqrBrphDcChF32",                                 
    app_secret="AK0WsjfbWrRMIL9aCi8b7i12hGdEojdohbUCCb++jDTe", 
    broker_id="SANDBOX",
    app_code="SANDBOX",
    is_auto_queue=False
)

deri = investor.Derivatives(account_no="Man-D")            

account_info = deri.get_account_info()

account_info    

{'creditLine': 2000000000.0,
 'excessEquity': 2000000000.0,
 'cashBalance': 2000000000.0,
 'equity': 2000000000.0,
 'totalMR': 0.0,
 'totalMM': 0.0,
 'totalFM': 0.0,
 'callForceFlag': 'No',
 'callForceMargin': 0.0,
 'liquidationValue': 2000000000.0,
 'depositWithdrawal': 0.0,
 'callForceMarginMM': 0.0,
 'initialMargin': 0.0,
 'closingMethod': 'Auto Net'}

In [4]:
#Get Candlestick
market = investor.MarketData()
res = market.get_candlestick(
symbol="AOT",
interval="1d",
limit=1,
normalized=True,)

In [5]:
if isinstance(res, dict) and "data" in res:
    raw_data = res["data"]
else:
    raw_data = res  # กรณี res เป็น list เลย
df = pd.DataFrame(raw_data)
df

,lastSequence,time,open,high,low,close,volume,value
0,0,1761843600,41.25,42.0,40.75,41.25,17948097,0


In [6]:
session.set_keyspace('stock_data')

In [7]:
# ✅ โหลดรายชื่อหุ้นทั้งหมด
import pandas as pd

try:
    symbols_df = pd.read_excel("//Users/hilmanyusoh/Desktop/Stock-market-chart-pattern-recognition-using-deep-learning/data_collection/settrade1.xlsx")
    symbols = symbols_df['หลักทรัพย์'].dropna().tolist()
except FileNotFoundError:
    print("❌ ไม่พบไฟล์ 'get_stock_name' กรุณาตรวจสอบชื่อไฟล์และตำแหน่งให้ถูกต้อง")
    raise  # หยุดโปรแกรมทันทีโดยแสดง error


# ✅ ฟังก์ชันดึง candlestick จาก API
def get_candlestick_data(symbol):
    try:
        res = market.get_candlestick(  # ✅ ต้องกำหนด market ก่อนใช้
            symbol=symbol,
            interval="1d",
            limit=1000,
            normalized=True
        )
        return res
    except Exception as e:
        print(f"❌ ไม่สามารถดึงข้อมูลของ {symbol}: {e}")
        return None

# ✅ ฟังก์ชัน insert ข้อมูลลง Cassandra
def insert_historical_data(symbol, res):
    if not res or 'time' not in res:
        print(f"⚠️ ไม่มีข้อมูลสำหรับ {symbol}")
        return

    for i in range(len(res["time"])):
        # ✅ แปลง timestamp ให้ถูกต้อง
        timestamp = datetime.fromtimestamp(res["time"][i])

        session.execute("""
            INSERT INTO candlestick_data (
                symbol, time, open_price, high_price, low_price,
                close_price, volume, value
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            symbol,
            timestamp,
            res["open"][i],
            res["high"][i],
            res["low"][i],
            res["close"][i],
            res["volume"][i],
            res["value"][i]
        ))

    print(f"✅ เพิ่มข้อมูล {len(res['time'])} รายการของหุ้น {symbol} สำเร็จ!")

# ✅ ดึงและบันทึกข้อมูลทุกหุ้น
for symbol in symbols:
    print(f"\n📦 กำลังโหลดข้อมูล {symbol} ...")
    res = get_candlestick_data(symbol)
    insert_historical_data(symbol, res)
    time.sleep(2)  # 🕒 ลดโอกาสถูก block จาก API

print("\n🎉 เสร็จสิ้นการโหลดข้อมูลทั้งหมด!")


📦 กำลังโหลดข้อมูล 24CS ...
✅ เพิ่มข้อมูล 1 รายการของหุ้น 24CS สำเร็จ!

📦 กำลังโหลดข้อมูล 2S ...
✅ เพิ่มข้อมูล 731 รายการของหุ้น 2S สำเร็จ!

📦 กำลังโหลดข้อมูล 3BBIF ...
✅ เพิ่มข้อมูล 731 รายการของหุ้น 3BBIF สำเร็จ!

📦 กำลังโหลดข้อมูล A ...
✅ เพิ่มข้อมูล 696 รายการของหุ้น A สำเร็จ!

📦 กำลังโหลดข้อมูล A5 ...
✅ เพิ่มข้อมูล 731 รายการของหุ้น A5 สำเร็จ!

📦 กำลังโหลดข้อมูล AAI ...
✅ เพิ่มข้อมูล 731 รายการของหุ้น AAI สำเร็จ!

📦 กำลังโหลดข้อมูล AAV ...
✅ เพิ่มข้อมูล 731 รายการของหุ้น AAV สำเร็จ!

📦 กำลังโหลดข้อมูล ABM ...
✅ เพิ่มข้อมูล 730 รายการของหุ้น ABM สำเร็จ!

📦 กำลังโหลดข้อมูล ACAP ...
✅ เพิ่มข้อมูล 100 รายการของหุ้น ACAP สำเร็จ!

📦 กำลังโหลดข้อมูล ACC ...
✅ เพิ่มข้อมูล 730 รายการของหุ้น ACC สำเร็จ!

📦 กำลังโหลดข้อมูล ACE ...
✅ เพิ่มข้อมูล 731 รายการของหุ้น ACE สำเร็จ!

📦 กำลังโหลดข้อมูล ACG ...
✅ เพิ่มข้อมูล 726 รายการของหุ้น ACG สำเร็จ!

📦 กำลังโหลดข้อมูล ADB ...
✅ เพิ่มข้อมูล 731 รายการของหุ้น ADB สำเร็จ!

📦 กำลังโหลดข้อมูล ADD ...
✅ เพิ่มข้อมูล 730 รายการของหุ้น ADD สำเร็จ!

📦 กำลัง

In [8]:
stock = "BTC"
rows = session.execute(f"SELECT * FROM candlestick_data WHERE symbol = '{stock}'")

In [9]:
df = pd.DataFrame(rows, columns=[
    'symbol', 'time', 'open_price', 'high_price', 'low_price', 'close_price', 'volume', 'value'
])

# ✅ แปลงชนิดเวลา + เรียงลำดับ
df['time'] = pd.to_datetime(df['time'])
df = df.sort_values('time').reset_index(drop=True)
df.head()

,symbol,time,open_price,high_price,low_price,close_price,volume,value
0,BTC,2022-10-17,0.45,0.48,0.45,0.47,0.0,22606675
1,BTC,2022-10-18,0.48,0.49,0.45,0.45,0.0,26635261
2,BTC,2022-10-19,0.47,0.49,0.47,0.48,0.0,7057878
3,BTC,2022-10-20,0.47,0.48,0.46,0.47,0.0,3966563
4,BTC,2022-10-21,0.47,0.48,0.46,0.47,0.0,4511270


In [10]:
import plotly.graph_objects as go

# ✅ สร้างกราฟ Candlestick
fig = go.Figure()

# 🔹 แท่งเทียนราคา
fig.add_trace(go.Candlestick(
    x=df['time'],
    open=df['open_price'],
    high=df['high_price'],
    low=df['low_price'],
    close=df['close_price'],
    name=stock
))

# 🔹 Volume bar (แถบปริมาณการซื้อขาย)
fig.add_trace(go.Bar(
    x=df['time'],
    y=df['volume'],
    marker_color='rgba(128,128,128,0.3)',
    yaxis='y2',
    name='Volume'
))

# ✅ ปรับ layout ให้เหมือน TradingView
fig.update_layout(
    title=f'📈 {stock} - Candlestick Chart with Volume',
    xaxis_rangeslider_visible=False,
    yaxis=dict(title='Price'),
    yaxis2=dict(
        title='Volume',
        overlaying='y',
        side='right',
        showgrid=False
    ),
    height=600,
    template='plotly_white'
)

fig.show()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/kaleido/__init__.py:14: UserWarning:




This means that static image generation (e.g. `fig.write_image()`) will not work.

Please upgrade Plotly to version 6.1.1 or greater, or downgrade Kaleido to version 0.2.1.


